In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import gensim
import wget
import numpy as np
from gensim.models import word2vec
from gensim.models import KeyedVectors
from keras.datasets import reuters
import numpy as np
import json
import warnings
import numpy as np
#from ..utils.data_utils import get_file
#from ..preprocessing.sequence import _remove_long_seq
import json
import warnings
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plot
#import plotly.offline as py
#import plotly.graph_objs as go
#py.init_notebook_mode(connected=True)
import matplotlib as plt
# Other
import re
import string
from sklearn.manifold import TSNE
from textblob import TextBlob
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Using TensorFlow backend.


<h1> Data Understanding </h1>

In [2]:
df = pd.read_csv('Combined_News_DJIA.csv')

In [3]:
df.tail(n=2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
1987,2016-06-30,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says pollution and 'stupidity'...,Boris Johnson says he will not run for Tory pa...,Six gay men in Ivory Coast were abused and for...,Switzerland denies citizenship to Muslim immig...,Palestinian terrorist stabs israeli teen girl ...,Puerto Rico will default on $1 billion of debt...,Republic of Ireland fans to be awarded medal f...,...,Googles free wifi at Indian railway stations i...,Mounting evidence suggests 'hobbits' were wipe...,The men who carried out Tuesday's terror attac...,Calls to suspend Saudi Arabia from UN Human Ri...,More Than 100 Nobel Laureates Call Out Greenpe...,British pedophile sentenced to 85 years in US ...,"US permitted 1,200 offshore fracks in Gulf of ...",We will be swimming in ridicule - French beach...,UEFA says no minutes of silence for Istanbul v...,Law Enforcement Sources: Gun Used in Paris Ter...
1988,2016-07-01,1,A 117-year-old woman in Mexico City finally re...,IMF chief backs Athens as permanent Olympic host,"The president of France says if Brexit won, so...",British Man Who Must Give Police 24 Hours' Not...,100+ Nobel laureates urge Greenpeace to stop o...,Brazil: Huge spike in number of police killing...,Austria's highest court annuls presidential el...,"Facebook wins privacy case, can track any Belg...",...,"The United States has placed Myanmar, Uzbekist...",S&amp;P revises European Union credit rating t...,India gets $1 billion loan from World Bank for...,U.S. sailors detained by Iran spoke too much u...,Mass fish kill in Vietnam solved as Taiwan ste...,Philippines president Rodrigo Duterte urges pe...,Spain arrests three Pakistanis accused of prom...,"Venezuela, where anger over food shortages is ...",A Hindu temple worker has been killed by three...,Ozone layer hole seems to be healing - US &amp...


<h1> Data Cleaning </h2>

In [4]:
def combineHeadlines(dataframe):
    testheadlines = []
    for row in range(0,len(dataframe.index)):
        testheadlines.append(' '.join(str(x) for x in dataframe.iloc[row,2:27]))         
    return testheadlines

In [5]:
data = combineHeadlines(df)

In [6]:
data = pd.DataFrame(data,columns=["text"])

In [7]:
#Script to cleaning text data
def clean_text(text):
    ## Remove punctuation
    #text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower().split()
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

<h1> Data Engineering</h1>

In [8]:
data['text'] = data['text'].map(lambda x: clean_text(x))

In [9]:
data['polarity'] = data['text'].map(lambda x: TextBlob(x).sentiment[0])

In [10]:
data['subjectivity'] = data['text'].map(lambda x: TextBlob(x).sentiment[1])

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(data['text'][0])['neg']
#print(str(vs))
#print(str(vs["compound"]))




0.263

In [12]:
data['neg'] = data['text'].map(lambda x: analyzer.polarity_scores(x)['neg'])
data['neu'] = data['text'].map(lambda x: analyzer.polarity_scores(x)['neu'])
data['pos'] = data['text'].map(lambda x: analyzer.polarity_scores(x)['pos'])
data['compound'] = data['text'].map(lambda x: analyzer.polarity_scores(x)['compound'])

In [13]:
data.tail(3)

,text,polarity,subjectivity,neg,neu,pos,compound
1986,explos airport istanbul yemeni former presid :...,0.122277,0.382513,0.188,0.714,0.098,-0.9818
1987,jamaica propos marijuana dispens tourist airpo...,0.047527,0.384982,0.235,0.683,0.083,-0.9963
1988,117 - year - old woman mexico citi final recei...,0.063571,0.299821,0.246,0.693,0.060,-0.9979


In [1]:
from polyglot.downloader import downloader
print(downloader.supported_languages_table("sentiment2", 3))

ModuleNotFoundError: No module named 'polyglot'

In [2]:
!pip install polyglot

  Using cached https://files.pythonhosted.org/packages/e9/35/211ffb949c68e688ade7d40426de030a24eaec4b6c45330eeb9c0285f43a/PyICU-2.3.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/21/77/8525fe5f147bf2819c7c9942c717c4a79b83f8003da1a3847759fb560909/pycld2-0.31.tar.gz
  Running setup.py bdist_wheel for PyICU ... error
  Complete output from command /anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/kj/fchz5ny54lbdzhb7z678qh940000gn/T/pip-install-5swm2igi/PyICU/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /private/var/folders/kj/fchz5ny54lbdzhb7z678qh940000gn/T/pip-wheel-ozw2efew --python-tag cp37:
  (running 'icu-config --version')
  
  Building PyICU 2.3.1 for ICU 58.2
  
  (running 'icu-config --cxxflags --cppflags')
  Adding CFLAGS="-I/anaconda3/include" from /anaconda3/bin/icu-config
  (running 'icu-config --ldflags')
  Addin

Command "/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/kj/fchz5ny54lbdzhb7z678qh940000gn/T/pip-install-5swm2igi/PyICU/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /private/var/folders/kj/fchz5ny54lbdzhb7z678qh940000gn/T/pip-record-g8ws2nw0/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /private/var/folders/kj/fchz5ny54lbdzhb7z678qh940000gn/T/pip-install-5swm2igi/PyICU/


In [14]:
df_list = [data,df[['Label','Date']]]
#pd.concat(df_list, ignore_index=True)
result = pd.concat(df_list,axis=1)

In [15]:
result.head(n=5)

,text,polarity,subjectivity,neg,neu,pos,compound,Label,Date
0,b georgia would own two russian warplan countr...,-0.031843,0.307501,0.263,0.690,0.047,-0.9970,0,2008-08-08
1,b whi wont america nato help us wont help now ...,0.024205,0.281621,0.247,0.691,0.063,-0.9906,1,2008-08-11
2,b are member ador 9 - year - old sang open cer...,-0.125450,0.488944,0.219,0.747,0.033,-0.9948,0,2008-08-12
3,u s refus israel weapon attack iran : report b...,-0.009991,0.266807,0.148,0.790,0.061,-0.9508,0,2008-08-13
4,b all expert admit legalis drug b war south os...,0.046292,0.275770,0.210,0.712,0.078,-0.9865,1,2008-08-14


In [16]:
train = result [result['Date']< '2015-01-01']
test = result[result['Date']> '2014-12-31']

In [17]:
train.shape

(1611, 9)

In [18]:
test.shape

(378, 9)

In [19]:
train.tail()

,text,polarity,subjectivity,neg,neu,pos,compound,Label,Date
1606,death toll among qatar 2022 world cup worker r...,-0.135877,0.285000,0.168,0.818,0.014,-0.9921,1,2014-12-24
1607,saudi eager await approv new draft law shoura ...,0.059042,0.370119,0.233,0.690,0.076,-0.9962,1,2014-12-26
1608,solar power storag price drop 25 germani lt;1 ...,0.062917,0.261736,0.232,0.710,0.057,-0.9947,0,2014-12-29
1609,china businessman jail year buy eat three tige...,0.037854,0.300318,0.173,0.778,0.050,-0.9914,0,2014-12-30
1610,airasia flight found bottom java sea north kor...,0.018524,0.324903,0.141,0.834,0.024,-0.9871,0,2014-12-31


<H1> MODELING </H1>

<h2>Word2Vec</h2>

In [18]:
filename = 'GoogleNews-vectors-negative300.bin'
#model = KeyedVectors.load_word2vec_format(filename, binary=True) what is the diff with below one?
#word2vec_path = "GoogleNews-vectors-negative300.bin"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=True)

In [20]:
columns = ['text','polarity','subjectivity','Label','Date','neg','neu','pos','compound']
X = train[columns]
Y = train["Label"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20,random_state=0)
y_train_t = pd.DataFrame(y_train,columns=["Label"])

y_train_t.head(n=5)

,Label
686,1
1535,1
751,0
96,1
178,1


In [21]:
tokenizer = RegexpTokenizer(r'\w+')
X_train["tokens"] = X_train["text"].apply(tokenizer.tokenize)

/Users/williamsuarez/anaconda2/envs/pandas_env/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
X_train.head(n=5)

,text,polarity,subjectivity,Label,Date,neg,neu,pos,compound,tokens
686,franc drop concret bomb libya tunisian armi en...,0.002997,0.351971,1,2011-04-29,0.221,0.727,0.051,-0.9959,"[franc, drop, concret, bomb, libya, tunisian, ..."
1535,former auschwitz guard 93 charg accessori 300 ...,-0.005350,0.290256,1,2014-09-15,0.133,0.825,0.041,-0.9834,"[former, auschwitz, guard, 93, charg, accessor..."
751,israel agre negoti pre - 67 line fatal radiat ...,0.031293,0.414103,0,2011-08-02,0.168,0.810,0.023,-0.9922,"[israel, agre, negoti, pre, 67, line, fatal, r..."
96,b hell earth citi shipbreak b full text irania...,0.162527,0.393398,1,2008-12-24,0.198,0.682,0.120,-0.9638,"[b, hell, earth, citi, shipbreak, b, full, tex..."
178,b i think peopl need understand waterboard is ...,0.052139,0.342647,1,2009-04-24,0.220,0.705,0.076,-0.9922,"[b, i, think, peopl, need, understand, waterbo..."


In [23]:
#max words
max_words = X_train['tokens'].apply(lambda x: len(x))
max(max_words)

499

In [24]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
# Set values for various parameters
num_features = 500    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(X_train['tokens'], workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-09-19 23:59:45,355 : INFO : collecting all words and their counts
2019-09-19 23:59:45,358 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-09-19 23:59:45,547 : INFO : collected 19677 word types from a corpus of 396584 raw words and 1288 sentences
2019-09-19 23:59:45,549 : INFO : Loading a fresh vocabulary


Training model...


2019-09-19 23:59:45,581 : INFO : effective_min_count=40 retains 1928 unique words (9% of original 19677, drops 17749)
2019-09-19 23:59:45,583 : INFO : effective_min_count=40 leaves 310545 word corpus (78% of original 396584, drops 86039)
2019-09-19 23:59:45,591 : INFO : deleting the raw counts dictionary of 19677 items
2019-09-19 23:59:45,596 : INFO : sample=0.001 downsamples 31 most-common words
2019-09-19 23:59:45,598 : INFO : downsampling leaves estimated 294014 word corpus (94.7% of prior 310545)
2019-09-19 23:59:45,614 : INFO : estimated required memory for 1928 words and 500 dimensions: 8676000 bytes
2019-09-19 23:59:45,615 : INFO : resetting layer weights
2019-09-19 23:59:45,666 : INFO : training model with 4 workers on 1928 vocabulary and 500 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2019-09-19 23:59:46,126 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-09-19 23:59:46,153 : INFO : worker thread finished; awaiting finish of 2 more threa

In [25]:
model.doesnt_match("canadian deep space".split())

/Users/williamsuarez/anaconda2/envs/pandas_env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/Users/williamsuarez/anaconda2/envs/pandas_env/lib/python2.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'canadian'

In [26]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%500. == 0.:
            print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [27]:
X_train_Vec = getAvgFeatureVecs(X_train['tokens'], model, num_features )

/Users/williamsuarez/anaconda2/envs/pandas_env/lib/python2.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 0 of 1288
Review 500 of 1288
Review 1000 of 1288


In [28]:
max_words = 500
num_classes = 1
model2 = Sequential()
model2.add(Dense(101, input_shape=(max_words,)))
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(num_classes))
model2.add(Activation('softmax'))

In [29]:
model2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [30]:
X_train_Vec.shape

(1288, 500)

In [31]:
X_train.head()

,text,polarity,subjectivity,Label,Date,neg,neu,pos,compound,tokens
686,franc drop concret bomb libya tunisian armi en...,0.002997,0.351971,1,2011-04-29,0.221,0.727,0.051,-0.9959,"[franc, drop, concret, bomb, libya, tunisian, ..."
1535,former auschwitz guard 93 charg accessori 300 ...,-0.005350,0.290256,1,2014-09-15,0.133,0.825,0.041,-0.9834,"[former, auschwitz, guard, 93, charg, accessor..."
751,israel agre negoti pre - 67 line fatal radiat ...,0.031293,0.414103,0,2011-08-02,0.168,0.810,0.023,-0.9922,"[israel, agre, negoti, pre, 67, line, fatal, r..."
96,b hell earth citi shipbreak b full text irania...,0.162527,0.393398,1,2008-12-24,0.198,0.682,0.120,-0.9638,"[b, hell, earth, citi, shipbreak, b, full, tex..."
178,b i think peopl need understand waterboard is ...,0.052139,0.342647,1,2009-04-24,0.220,0.705,0.076,-0.9922,"[b, i, think, peopl, need, understand, waterbo..."


In [32]:
y_train_t.shape

(1288, 1)

In [ ]:
batch_size = 100
epochs = 2
history = model2.fit(X_train_Vec, np.array(y_train_t['Label']), batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)

In [26]:
score = model2.evaluate(datatest, y_test, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'datatest' is not defined

In [33]:
XtrainVec =  X_train['text'].as_matrix()

/Users/williamsuarez/anaconda2/envs/pandas_env/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [34]:
XtestVec =  X_test['text'].as_matrix()

/Users/williamsuarez/anaconda2/envs/pandas_env/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [35]:
XtrainVec.shape

(1288,)

In [36]:
advanceVectorizer = CountVectorizer(ngram_range=(2,2))
advancedtrain = advanceVectorizer.fit_transform(XtrainVec)
#advancedtest = advanceVectorizer.transform(XtestVec)

In [37]:
advancedtrain.shape
advancedtrain= advancedtrain.todense()

In [38]:
new_advanced_train=pd.DataFrame(data=advancedtrain[0:,0:],
                   index=[i for i in range(advancedtrain.shape[0])],
                   columns=[''+str(i) for i in range(advancedtrain.shape[1])])

In [39]:
new_advanced_train.head(2)

,0,1,2,3,4,5,6,7,8,9,...,283814,283815,283816,283817,283818,283819,283820,283821,283822,283823
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
test_dataset = advanceVectorizer.transform(XtestVec)
advancedtest= test_dataset.todense()
new_advanced_test=pd.DataFrame(data=advancedtest[0:,0:],
                   index=[i for i in range(advancedtest.shape[0])],
                   columns=[''+str(i) for i in range(advancedtest.shape[1])])
#X_train['textVec'] = list(advancedtrain.toarray()) 

In [41]:
print(new_advanced_train.shape)
print(new_advanced_test.shape)


(1288, 283824)
(323, 283824)


In [42]:
new_train = pd.DataFrame()
new_train['polarity'] = X_train['polarity']
new_train['subjectivity'] = X_train['subjectivity']
new_train['neg'] = X_train['neg']
new_train['neu'] = X_train['neu']
new_train['pos'] = X_train['pos']
new_train['compound'] = X_train['compound']
new_test = pd.DataFrame()
new_test['polarity'] = X_test['polarity']
new_test['subjectivity'] = X_test['subjectivity']
new_test['neg'] = X_test['neg']
new_test['neu'] = X_test['neu']
new_test['pos'] = X_test['pos']
new_test['compound'] = X_test['compound']

In [43]:
new_train = new_train.reset_index()
new_test = new_test.reset_index()

In [44]:
new_train.head()

,index,polarity,subjectivity,neg,neu,pos,compound
0,686,0.002997,0.351971,0.221,0.727,0.051,-0.9959
1,1535,-0.005350,0.290256,0.133,0.825,0.041,-0.9834
2,751,0.031293,0.414103,0.168,0.810,0.023,-0.9922
3,96,0.162527,0.393398,0.198,0.682,0.120,-0.9638
4,178,0.052139,0.342647,0.220,0.705,0.076,-0.9922


In [45]:
new_advanced_train['polarity'] = new_train['polarity']
new_advanced_train['subjectivity'] = new_train['subjectivity']
new_advanced_train['neg'] = new_train['neg']
new_advanced_train['neu'] = new_train['neu']
new_advanced_train['pos'] = new_train['pos']
new_advanced_train['compound'] = new_train['compound']

new_advanced_test['polarity'] = new_test['polarity']
new_advanced_test['subjectivity'] = new_test['subjectivity']
new_advanced_test['neg'] = new_test['neg']
new_advanced_test['neu'] = new_test['neu']
new_advanced_test['pos'] = new_test['pos']
new_advanced_test['compound'] = new_test['compound']

In [46]:
print(new_advanced_train.shape)
print(new_advanced_test.shape)

(1288, 283830)
(323, 283830)


In [47]:
#X_test['textVec'] = list(test_dataset.toarray()) 

In [48]:
#X_test.head(2)

In [67]:
randomforest = RandomForestClassifier(n_estimators = 200,criterion='entropy',oob_score = True, n_jobs = -1,random_state =50,max_features = "auto", min_samples_leaf = 50)
randomforest = randomforest.fit(new_advanced_train, y_train_t["Label"])
predictions = randomforest.predict(new_advanced_test)

In [68]:
y_test_t = pd.DataFrame(y_test,columns=["Label"])
score = accuracy_score(y_test_t['Label'],predictions)

In [69]:
print("Accuracy: %.2f%%" % (score * 100.0))

Accuracy: 58.82%


In [58]:
columns = ['polarity','subjectivity','neg','neu','pos','compound']
new_advanced_train2= new_advanced_train[columns]
new_advanced_test2= new_advanced_test[columns]

In [59]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)
xgb_model = xgb_model.fit(new_advanced_train2, y_train_t["Label"])
advancedprediction = xgb_model.predict(new_advanced_test2)

In [60]:
predictions = [round(value) for value in advancedprediction]

In [61]:
#evaluate predictions
accuracy = accuracy_score(y_test_t['Label'], predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 54.18%


In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)
xgb_model = xgb_model.fit(new_advanced_train, y_train_t["Label"])
advancedprediction = xgb_model.predict(new_advanced_test)

/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



[16:31:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [173]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)
xgb_model = xgb_model.fit(X_train_polarity, y_train_t["Label"])

[15:37:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [174]:
advancedprediction = xgb_model.predict(X_test_polarity)

In [175]:
predictions = [round(value) for value in advancedprediction]

In [176]:
#evaluate predictions
accuracy = accuracy_score(y_test_t['Label'], predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 54.49%


In [206]:
randomforest = RandomForestClassifier(n_estimators = 200,criterion='entropy')
randomforest = randomforest.fit(X_train_polarity, y_train_t["Label"])
predictions = randomforest.predict(X_test_polarity)

In [207]:
score = accuracy_score(y_test_t['Label'],predictions)

In [208]:
print("Accuracy: %.2f%%" % (score * 100.0))

Accuracy: 54.80%
